In [ ]:
import json
import os
token = {"username":"vinaybhupalam","key":"5c27e8b73e30502d96da3d5b41a37fed"}
os.mkdir('/root/.kaggle')
with open('/root/.kaggle/kaggle.json', 'w+') as file:
    json.dump(token, file)

In [ ]:
import os
import shutil
import glob
import keras
from skimage import io
from skimage.transform import resize
import os
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

import warnings
warnings.simplefilter('ignore')
print("DOne")

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
!ls

In [ ]:
!unzip chest-xray-pneumonia.zip

In [ ]:
!ls

In [ ]:
DATASET_PATH = '/content/chest_xray'

# There are two classes of images that we will deal with
disease_cls = ['NORMAL' ,'PNEUMONIA'              
              ]

In [ ]:
!ls

In [ ]:
import glob
for index,class_type in enumerate(disease_cls):
    path = os.path.join(DATASET_PATH, 'train', disease_cls[index], '*')
    count_datapoints = len(glob.glob(path))
    print("Number of data points for Label {}  is {}" .format(count_datapoints,class_type))


Normal_path = os.path.join(DATASET_PATH, 'train', disease_cls[0], '*')
Normal = glob.glob(Normal_path)
Normal = io.imread(Normal[0],True)
Pnuemonia_path = os.path.join(DATASET_PATH, 'train', disease_cls[1], '*')
Pnuemonia = glob.glob(Pnuemonia_path)
Pnuemonia = io.imread(Pnuemonia[0],True)

print(Pnuemonia.shape)
f, axes = plt.subplots(1, 2, sharey=True)
f.set_figwidth(10)
    
axes[0].imshow(Normal, cmap='gray')
axes[1].imshow(Pnuemonia, cmap='gray')


image_resized_Normal = resize(Normal, (256, 256))
image_resized_Pneumonia = resize(Pnuemonia, (256, 256))
f, axes = plt.subplots(1, 2, sharey=True)
f.set_figwidth(10)
axes[0].imshow(image_resized_Normal, cmap='gray')
axes[1].imshow(image_resized_Pneumonia, cmap='gray')

image_resized_Normal.shape

In [ ]:
from skimage.transform import rescale
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
    width_shift_range=0,
    height_shift_range=0,
    vertical_flip=False,)

def preprocess_img(img, mode):
    img = (img - img.min())/(img.max() - img.min())
    
    img = resize(img, (256, 256))
    
    if mode == 'train':
        if np.random.randn() > 0:
            img = datagen.random_transform(img)
    return img

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import resnet_modified
#model = resnet_modified.ResnextBuilder.build_resnet_18((img_channels, img_rows, img_cols), nb_classes)
from google.colab import files
upload= files.upload()

In [ ]:
import resnet_modified

img_channels = 1
img_rows = 256
img_cols = 256

nb_classes = 2

In [ ]:
import numpy as np
import keras

class AugmentedDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, mode='train', ablation=None, disease_cls = ['NORMAL', 'PNEUMONIA'], 
                 batch_size=32, dim=(256, 256), n_channels=1, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = {}
        self.list_IDs = []
        self.mode = mode
        
        for i, cls in enumerate(disease_cls):
            print(cls)
            paths = glob.glob(os.path.join(DATASET_PATH,'train', cls, '*')) 
            brk_point = int(len(paths)*0.8)           
            if self.mode == 'train':         
                paths = paths[:brk_point]
            elif self.mode == 'validation':
                paths = paths[brk_point:]
            elif self.mode == 'test':
                paths = glob.glob(os.path.join(DATASET_PATH, "test", cls, '*'))    
            if ablation is not None:
                paths = paths[:int(len(paths)*ablation/100)]
            print(len(paths) ,cls,self.mode)
            self.list_IDs += paths
            self.labels.update({p:i for p in paths})
        
            
        self.n_channels = n_channels
        self.n_classes = len(disease_cls)
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'

        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        
        delete_rows = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            img = io.imread(ID,True)
            img = img[:, :, np.newaxis]            
            img = preprocess_img(img, self.mode)
            X[i,] = img
            y[i] = self.labels[ID]         
              
                
        X = np.delete(X, delete_rows, axis=0)
        y = np.delete(y, delete_rows, axis=0)
        # from sklearn.utils import class_weight
        # global class_weight
        # class_weight = class_weight.compute_class_weight('balanced', np.unique(y), y)
        # print(class_weight)
        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:

model = resnet_modified.ResnextBuilder.build_resnet_18((img_channels, img_rows, img_cols), nb_classes,3)
from keras import optimizers
sgd_momentum = optimizers.SGD(lr=0.005 ,momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd_momentum,
              metrics=['accuracy'])
training_generator = AugmentedDataGenerator('train', ablation=20 ,disease_cls=disease_cls)
validation_generator = AugmentedDataGenerator('validation', ablation=20,disease_cls = disease_cls)

model.fit_generator(generator=training_generator,
                    validation_data=None,
                    epochs=5)

In [ ]:
class DecayLR(keras.callbacks.Callback):
    def __init__(self, base_lr=0.01, decay_epoch=1):
        super(DecayLR, self).__init__()
        self.base_lr = base_lr
        self.decay_epoch = decay_epoch 
        self.lr_history = []
        
    def on_train_begin(self, logs={}):
        K.set_value(self.model.optimizer.lr, self.base_lr)

    def on_epoch_end(self, epoch, logs={}):
        new_lr = self.base_lr * (0.5 ** (epoch // self.decay_epoch))
        self.lr_history.append(K.get_value(self.model.optimizer.lr))
        K.set_value(self.model.optimizer.lr, new_lr)

In [ ]:
from sklearn.metrics import roc_auc_score, recall_score
from keras import optimizers
from keras.callbacks import *

class roc_callback(Callback):
    
    def on_train_begin(self, logs={}):
        pass

    def on_epoch_end(self, epoch, logs={}):
        y_p = []
        y_v = []
        for i in range(len(validation_generator)):
            x_val, y_val = validation_generator[i]
            y_pred = self.model.predict(x_val)
            y_p.append(y_pred)
            y_v.append(y_val)
        y_p = np.concatenate(y_p)
        y_v = np.concatenate(y_v)
        roc_auc = roc_auc_score(y_v, y_p)        
        print ('\nVal AUC for epoch{}: {}'.format(epoch, roc_auc))
        logs['val_auc'] = roc_auc

In [ ]:
model = resnet_modified.ResnextBuilder.build_resnet_18((img_channels, img_rows, img_cols), nb_classes,3)
sgd = optimizers.SGD(lr=0.01,momentum=0.9)

model.compile(loss='categorical_crossentropy',optimizer= sgd,
              metrics=['accuracy'])
training_generator = AugmentedDataGenerator('train', ablation=100)
validation_generator = AugmentedDataGenerator('validation', ablation=100)

auc_logger = roc_callback()
decay = DecayLR()

model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=7, callbacks=[auc_logger, decay])